<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Deciding Where to Open a Pizza Place in Seattle</font></h1>

# Step 1 - Scrape, Clean, and Organize Data

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
#Install BeautifulSoup4 for web scraping
!pip install beautifulsoup4

!pip install lxml

     |████████████████████████████████| 112kB 10.0MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 29.9MB/s eta 0:00:01


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    numpy-1.17.3               |   py36h95a1406_0         5.2 MB  conda-forge
    sci

<a id='item1'></a>

## Download and Explore Dataset

In [3]:
#Use BeautifulSoup4 to scrape the wikipedia page
#Import requests
import requests

#Set website_url to the list of postal codes in Toronto
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Seattle').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html')
#print(soup.prettify())

In [4]:
#Using bs4, load the data into a Pandas dataframe
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
neighborhood_data = pd.DataFrame(data = data,columns = columns)
neighborhood_data.head()

,,Neighborhood name,Within larger district,Annexed[41],Locator map,Street map,Image,Notes
0,1,North Seattle,Seattle,Various,,,,North of the Lake Washington Ship Canal[42]
1,2,Broadview,North Seattle[42],1954[43],,,,[44]
2,3,Bitter Lake,North Seattle[42],1954[43],,,,[45]
3,4,North Beach / Blue Ridge,North Seattle[42],"1940,[43] 1954[43]",,,,[46]
4,5,Crown Hill,North Seattle[42],"1907,[47] 1952,[43] 1954[43]",,,,[48]


In [5]:
#Set index as 'Neighborhood name'
neighborhood_data = neighborhood_data.set_index('')
neighborhood_data.head()

,Neighborhood name,Within larger district,Annexed[41],Locator map,Street map,Image,Notes
,,,,,,,
1,North Seattle,Seattle,Various,,,,North of the Lake Washington Ship Canal[42]
2,Broadview,North Seattle[42],1954[43],,,,[44]
3,Bitter Lake,North Seattle[42],1954[43],,,,[45]
4,North Beach / Blue Ridge,North Seattle[42],"1940,[43] 1954[43]",,,,[46]
5,Crown Hill,North Seattle[42],"1907,[47] 1952,[43] 1954[43]",,,,[48]


In [6]:
#Drop 'Within larger district', 'Annexed', Locator map, Street map, Image, and Notes
neighborhood_data = neighborhood_data.drop(columns=['Within larger district','Annexed[41]','Locator map','Street map','Image','Notes'])
neighborhood_data.head()

,Neighborhood name
,
1,North Seattle
2,Broadview
3,Bitter Lake
4,North Beach / Blue Ridge
5,Crown Hill


After attempting to find a table online with all neighborhoods and Latitude and Longitude values, I decided to download the file as a .csv file and manually put together a table with all of the needed information

In [7]:
neighborhood_data.to_csv("neighborhood_data.csv", index=False)

#### Load and explore the data

Next, let's load the data for all Seattle neighborhoods, including Latitude and Longitude values.

In [8]:
geospatial_data = pd.read_csv("seattle_neighborhood_location.csv")

geospatial_data.head()

,Neighborhood name,Latitude,Longitude
0,Alki Point,47.5755,-122.4107
1,Arbor Heights,47.5095,-122.3812
2,Ballard,47.6792,-122.3860
3,Beacon Hill,47.5526,-122.3009
4,Belltown,47.6417,-122.3448


Let's take a quick look at the shape of the data.

In [9]:
#Use the pandas 'shape' function to look at the size of the dataframe
geospatial_data.shape

(107, 3)

In [10]:
#Let's find the average Latitude and Longitude to find a good centerpoint to begin creating a map using Folium
print('The average Latitude is: ', geospatial_data['Latitude'].mean())
print('The average Longitude is: ', geospatial_data['Longitude'].mean())

The average Latitude is:  47.63054485981308
The average Longitude is:  -122.31491495327103


#### Create a map of Seattle neighborhoods to visualize the data points

In [11]:
#Define Toronto Latitude & Longitude
Latitude = 47.6044
Longitude = -122.2441


# create map of New York using latitude and longitude values
map_seattle = folium.Map(location=[Latitude, Longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(geospatial_data['Latitude'], geospatial_data['Longitude'], geospatial_data['Neighborhood name']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_seattle)  
    
map_seattle

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'IHKHEVAA04WSWPXRX5OWVXNRE2LAUO44TK2L1SLPB4LP5AUE' # your Foursquare ID
CLIENT_SECRET = 'DYZGBPHDTNQHT4E52054AMG5H4BT4HJGR1A0F3GREJ3ZMPSG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IHKHEVAA04WSWPXRX5OWVXNRE2LAUO44TK2L1SLPB4LP5AUE
CLIENT_SECRET:DYZGBPHDTNQHT4E52054AMG5H4BT4HJGR1A0F3GREJ3ZMPSG


<a id='item2'></a>

## Explore Neighborhoods in Seattle

#### Let's create a function to repeat the same process to all the neighborhoods in Seattle

In [13]:
LIMIT = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [14]:
# type your answer here

seattle_venues = getNearbyVenues(names=geospatial_data['Neighborhood name'],
                                   latitudes=geospatial_data['Latitude'],
                                   longitudes=geospatial_data['Longitude']
                                  )



Alki Point
Arbor Heights
Ballard
Beacon Hill
Belltown
Bitter Lake
Briarcliff
Brighton
Broadmoor
Broadview
Bryant
Capitol Hill
Cascade, Seattle
Cedar Park
Central Seattle
Central Waterfront
Cherry Hill & Squire Park
Columbia City
Crown Hill
Delridge
Denny Triangle
Dunlap / Othello
East Queen Anne
Eastlake
Fairmount Park
Fauntleroy
First Hill
Fremont
Gatewood
Genesee
Georgetown
Green Lake
Haller Lake
Harbor Island
Harrison[120] / Denny-Blaine[120]
Hawthorne Hills
High Point
Highland Park
Hillman City
Holly Park[144] / NewHolly[145]
Industrial District
Interbay
International District ("ID")
Judkins Park
Junction[154] / West Seattle Junction / Alaska Junction
Lake City
Lake Union
Laurelhurst
Lawton Park
Leschi[120]
Lower Queen Anne
Loyal Heights
Madison Park
Madison Valley
Madrona
Magnolia
Mann
Maple Leaf
Matthews Beach
Meadowbrook
Montlake
North Admiral[153] / Admiral District
North Beacon Hill
North College Park
(Licton Springs)
North Delridge
North Queen Anne
North Seattle
Northgate
Nor

#### Let's check the size of the resulting dataframe

In [15]:
print(seattle_venues.shape)
seattle_venues.head()

(2433, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alki Point,47.5755,-122.4107,Constellation Park,47.573766,-122.415949,Beach
1,Alki Point,47.5755,-122.4107,Cactus,47.578234,-122.413106,Mexican Restaurant
2,Alki Point,47.5755,-122.4107,Pegasus Pizza,47.578722,-122.411816,Pizza Place
3,Alki Point,47.5755,-122.4107,Alki Beach Neighborhood,47.579450,-122.409303,Neighborhood
4,Alki Point,47.5755,-122.4107,Starbucks,47.578838,-122.411199,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [16]:
seattle_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alki Point,35,35,35,35,35,35
Arbor Heights,2,2,2,2,2,2
Ballard,5,5,5,5,5,5
Beacon Hill,2,2,2,2,2,2
Belltown,22,22,22,22,22,22
Bitter Lake,31,31,31,31,31,31
Briarcliff,3,3,3,3,3,3
Brighton,11,11,11,11,11,11
Broadmoor,4,4,4,4,4,4


#### Let's find out how many unique categories can be curated from all the returned venues

In [17]:
print('There are {} uniques categories.'.format(len(seattle_venues['Venue Category'].unique())))

There are 308 uniques categories.


<a id='item3'></a>

## Analyze Each Neighborhood

In [18]:
# one hot encoding
seattle_onehot = pd.get_dummies(seattle_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
seattle_onehot['Neighborhood'] = seattle_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [seattle_onehot.columns[-1]] + list(seattle_onehot.columns[:-1])
seattle_onehot = seattle_onehot[fixed_columns]

seattle_onehot.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Shop,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Parking,Pawn Shop,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Pub,Public Art,Radio Station,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Rugby Pitch,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Toy / Game

And let's examine the new dataframe size.

In [19]:
seattle_onehot.shape

(2433, 308)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [20]:
seattle_grouped = seattle_onehot.groupby('Neighborhood').mean().reset_index()
seattle_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Shop,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eye Doctor,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Parking,Pawn Shop,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Pub,Public Art,Radio Station,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Rugby Pitch,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Plaza,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Toy / Game

#### Let's confirm the new size

In [21]:
seattle_grouped.shape

(105, 308)

#### Let's print each neighborhood along with the top 5 most common venues

In [22]:
num_top_venues = 5

for hood in seattle_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = seattle_grouped[seattle_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alki Point----
                venue  freq
0         Coffee Shop  0.09
1                Park  0.06
2         Art Gallery  0.06
3  Italian Restaurant  0.06
4          Restaurant  0.06


----Arbor Heights----
                     venue  freq
0                     Park   0.5
1              Music Venue   0.5
2              Yoga Studio   0.0
3  New American Restaurant   0.0
4          Organic Grocery   0.0


----Ballard----
               venue  freq
0               Park   0.2
1           Bus Stop   0.2
2        Coffee Shop   0.2
3  French Restaurant   0.2
4      Jewelry Store   0.2


----Beacon Hill----
                venue  freq
0  Light Rail Station   0.5
1         Bus Station   0.5
2         Yoga Studio   0.0
3           Nightclub   0.0
4     Organic Grocery   0.0


----Belltown----
             venue  freq
0         Bus Stop  0.23
1  Harbor / Marina  0.18
2    Boat or Ferry  0.14
3        Dive Shop  0.05
4      Coffee Shop  0.05


----Bitter Lake----
                   venue  freq

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
seattle_venues_sorted = pd.DataFrame(columns=columns)
seattle_venues_sorted['Neighborhood'] = seattle_grouped['Neighborhood']

for ind in np.arange(seattle_grouped.shape[0]):
    seattle_venues_sorted.iloc[ind, 1:] = return_most_common_venues(seattle_grouped.iloc[ind, :], num_top_venues)

seattle_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alki Point,Coffee Shop,Restaurant,Park,Art Gallery,Beach,Italian Restaurant,Baseball Field,Baseball Stadium,French Restaurant,Donut Shop
1,Arbor Heights,Music Venue,Park,Event Space,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Women's Store
2,Ballard,French Restaurant,Bus Stop,Park,Jewelry Store,Coffee Shop,Women's Store,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
3,Beacon Hill,Bus Station,Light Rail Station,Women's Store,Eye Doctor,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fair
4,Belltown,Bus Stop,Harbor / Marina,Boat or Ferry,Intersection,Deli / Bodega,Dive Shop,Bus Line,Park,Trail,Grocery Store


<a id='item4'></a>

## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [25]:
# set number of clusters
kclusters = 5

seattle_grouped_clustering = seattle_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(seattle_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 0, 0, 0, 2, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [26]:
# add clustering labels
seattle_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

seattle_merged = geospatial_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
seattle_merged = seattle_merged.join(seattle_venues_sorted.set_index('Neighborhood'), on='Neighborhood name')

In [27]:
seattle_merged.head()

,Neighborhood name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alki Point,47.5755,-122.4107,0.0,Coffee Shop,Restaurant,Park,Art Gallery,Beach,Italian Restaurant,Baseball Field,Baseball Stadium,French Restaurant,Donut Shop
1,Arbor Heights,47.5095,-122.3812,2.0,Music Venue,Park,Event Space,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Women's Store
2,Ballard,47.6792,-122.3860,0.0,French Restaurant,Bus Stop,Park,Jewelry Store,Coffee Shop,Women's Store,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
3,Beacon Hill,47.5526,-122.3009,0.0,Bus Station,Light Rail Station,Women's Store,Eye Doctor,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fair
4,Belltown,47.6417,-122.3448,0.0,Bus Stop,Harbor / Marina,Boat or Ferry,Intersection,Deli / Bodega,Dive Shop,Bus Line,Park,Trail,Grocery Store


In [28]:
seattle_merged = seattle_merged.dropna()

In [29]:
seattle_merged.dtypes

Neighborhood name          object
Latitude                  float64
Longitude                 float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [30]:
seattle_merged.head()

,Neighborhood name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alki Point,47.5755,-122.4107,0.0,Coffee Shop,Restaurant,Park,Art Gallery,Beach,Italian Restaurant,Baseball Field,Baseball Stadium,French Restaurant,Donut Shop
1,Arbor Heights,47.5095,-122.3812,2.0,Music Venue,Park,Event Space,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Women's Store
2,Ballard,47.6792,-122.3860,0.0,French Restaurant,Bus Stop,Park,Jewelry Store,Coffee Shop,Women's Store,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
3,Beacon Hill,47.5526,-122.3009,0.0,Bus Station,Light Rail Station,Women's Store,Eye Doctor,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fair
4,Belltown,47.6417,-122.3448,0.0,Bus Stop,Harbor / Marina,Boat or Ferry,Intersection,Deli / Bodega,Dive Shop,Bus Line,Park,Trail,Grocery Store


In [31]:
seattle_merged['Cluster Labels'] = seattle_merged['Cluster Labels'].astype(np.int64)

In [32]:
seattle_merged.dtypes

Neighborhood name          object
Latitude                  float64
Longitude                 float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [33]:
# create map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(seattle_merged['Latitude'], seattle_merged['Longitude'], seattle_merged['Neighborhood name'], seattle_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [34]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 0, seattle_merged.columns[[0] + list(range(4, seattle_merged.shape[1]))]]

,Neighborhood name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alki Point,Coffee Shop,Restaurant,Park,Art Gallery,Beach,Italian Restaurant,Baseball Field,Baseball Stadium,French Restaurant,Donut Shop
2,Ballard,French Restaurant,Bus Stop,Park,Jewelry Store,Coffee Shop,Women's Store,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
3,Beacon Hill,Bus Station,Light Rail Station,Women's Store,Eye Doctor,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fair
4,Belltown,Bus Stop,Harbor / Marina,Boat or Ferry,Intersection,Deli / Bodega,Dive Shop,Bus Line,Park,Trail,Grocery Store
5,Bitter Lake,Automotive Shop,Marijuana Dispensary,Thai Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Food Truck,Beer Bar,Intersection,Sandwich Place,Chinese Restaurant
7,Brighton,Mexican Restaurant,Vietnamese Restaurant,Food,Seafood Restaurant,Gas Station,Gym,Fast Food Restaurant,Coffee Shop,Pizza Place,Coworking Space
8,Broadmoor,Soccer Field,Playground,Garden,Golf Course,Ethiopian Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
9,Broadview,Concert Hall,Trail,Dog Run,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
10,Bryant,Trail,Farm,Baseball Field,Women's Store,Event Space,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
11,Capitol Hill,Cocktail Bar,Coffee Shop,American Restaurant,Spa,Café,Restaurant,Thai Restaurant,Bar,Italian Restaurant,Donut Shop


#### Cluster 2

In [35]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 1, seattle_merged.columns[[0] + list(range(4, seattle_merged.shape[1]))]]

,Neighborhood name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,View Ridge,Tennis Court,Women's Store,Eye Doctor,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fair


#### Cluster 3

In [36]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 2, seattle_merged.columns[[0] + list(range(4, seattle_merged.shape[1]))]]

,Neighborhood name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Arbor Heights,Music Venue,Park,Event Space,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Women's Store
6,Briarcliff,Scenic Lookout,Park,Beach,Women's Store,Ethiopian Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
22,East Queen Anne,Playground,Park,Women's Store,Ethiopian Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
34,Harrison[120] / Denny-Blaine[120],Park,Monument / Landmark,Beach,Women's Store,Eye Doctor,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
43,Judkins Park,Park,Scenic Lookout,Bus Stop,Trail,Tunnel,Museum,Skate Park,Donut Shop,Drugstore,Dry Cleaner
46,Lake Union,Park,Harbor / Marina,Grocery Store,Women's Store,Ethiopian Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant
58,Matthews Beach,Playground,Vacation Rental,Trail,Park,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
63,North College Park\n(Licton Springs),Community College,Playground,Park,Food Court,Women's Store,Event Space,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant
81,Renton Hill,Other Repair Shop,Park,Italian Restaurant,Dog Run,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
86,South Beacon Hill[146] / Van Asselt,Park,Grocery Store,Women's Store,Dog Run,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant


#### Cluster 4

In [37]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 3, seattle_merged.columns[[0] + list(range(4, seattle_merged.shape[1]))]]

,Neighborhood name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
105,Windermere,Pizza Place,Women's Store,Eye Doctor,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space


#### Cluster 5

In [38]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 4, seattle_merged.columns[[0] + list(range(4, seattle_merged.shape[1]))]]

,Neighborhood name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,High Point,Trail,Women's Store,Dog Run,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space


# Breaking Down Cluster 1

Cluster 1 clearly has venues that would correlate to a successful pizza restaurant. Now I will take Cluster 1 and narrow it down further to see which neighborhood stands out as the best option. To do this, I will eliminate any neighborhood that has "Pizza Place" as a top 10 most common venue.

In [43]:
pizza_venues = seattle_merged.loc[seattle_merged['Cluster Labels'] == 0, seattle_merged.columns[[0] + list(range(4, seattle_merged.shape[1]))]]
pizza_venues.set_index('Neighborhood name')
pizza_venues.head()

,Neighborhood name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alki Point,Coffee Shop,Restaurant,Park,Art Gallery,Beach,Italian Restaurant,Baseball Field,Baseball Stadium,French Restaurant,Donut Shop
2,Ballard,French Restaurant,Bus Stop,Park,Jewelry Store,Coffee Shop,Women's Store,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
3,Beacon Hill,Bus Station,Light Rail Station,Women's Store,Eye Doctor,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fair
4,Belltown,Bus Stop,Harbor / Marina,Boat or Ferry,Intersection,Deli / Bodega,Dive Shop,Bus Line,Park,Trail,Grocery Store
5,Bitter Lake,Automotive Shop,Marijuana Dispensary,Thai Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Food Truck,Beer Bar,Intersection,Sandwich Place,Chinese Restaurant


In [44]:
pizza_venues.shape

(91, 11)

In [45]:
#Drop "Pizza Place" from the first 5 most common venues
pizza_venues = pizza_venues[pizza_venues['1st Most Common Venue'] !='Pizza Place']
pizza_venues = pizza_venues[pizza_venues['2nd Most Common Venue'] !='Pizza Place']
pizza_venues = pizza_venues[pizza_venues['3rd Most Common Venue'] !='Pizza Place']
pizza_venues = pizza_venues[pizza_venues['4th Most Common Venue'] !='Pizza Place']
pizza_venues = pizza_venues[pizza_venues['5th Most Common Venue'] !='Pizza Place']
pizza_venues = pizza_venues[pizza_venues['6th Most Common Venue'] !='Pizza Place']
pizza_venues = pizza_venues[pizza_venues['7th Most Common Venue'] !='Pizza Place']
pizza_venues = pizza_venues[pizza_venues['8th Most Common Venue'] !='Pizza Place']
pizza_venues = pizza_venues[pizza_venues['9th Most Common Venue'] !='Pizza Place']
pizza_venues = pizza_venues[pizza_venues['10th Most Common Venue'] !='Pizza Place']

pizza_venues.shape

(66, 11)

In [46]:
pizza_venues.head()

,Neighborhood name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alki Point,Coffee Shop,Restaurant,Park,Art Gallery,Beach,Italian Restaurant,Baseball Field,Baseball Stadium,French Restaurant,Donut Shop
2,Ballard,French Restaurant,Bus Stop,Park,Jewelry Store,Coffee Shop,Women's Store,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
3,Beacon Hill,Bus Station,Light Rail Station,Women's Store,Eye Doctor,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fair
4,Belltown,Bus Stop,Harbor / Marina,Boat or Ferry,Intersection,Deli / Bodega,Dive Shop,Bus Line,Park,Trail,Grocery Store
5,Bitter Lake,Automotive Shop,Marijuana Dispensary,Thai Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Food Truck,Beer Bar,Intersection,Sandwich Place,Chinese Restaurant


To narrow down the options further, I analyzed the different types of food venues that appear in the dataset. Based on personal experience, I decided to choose the top 10 types of venues that would compete against a Pizza Place. These venues are 'American Restaurant', 'BBQ Joint', 'Burger Joint', 'Chinese Restaurant', 'Italian Restaurant', 'Latin American Restaurant', 'New American Restaurant', 'Sandwich Place', 'Sports Bar', and 'Steakhouse'. I will eliminate any neighborhood that has these venues within their top 5 most common venues. The idea behind this is to introduce a new Pizza Place to an area where it can out-compete the current venues in the area. 

In [52]:
#Drop Competeing Venues from '1st Most Common Venue'
pizza_venues = pizza_venues[pizza_venues['1st Most Common Venue'] !='American Restaurant']
pizza_venues = pizza_venues[pizza_venues['1st Most Common Venue'] !='BBQ Joint']
pizza_venues = pizza_venues[pizza_venues['1st Most Common Venue'] !='Burger Joint']
pizza_venues = pizza_venues[pizza_venues['1st Most Common Venue'] !='Chinese Restaurant']
pizza_venues = pizza_venues[pizza_venues['1st Most Common Venue'] !='Italian Restaurant']
pizza_venues = pizza_venues[pizza_venues['1st Most Common Venue'] !='Latin American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['1st Most Common Venue'] !='New American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['1st Most Common Venue'] !='Sandwich Place']
pizza_venues = pizza_venues[pizza_venues['1st Most Common Venue'] !='Sports Bar']
pizza_venues = pizza_venues[pizza_venues['1st Most Common Venue'] !='Steakhouse']

#Drop Competeing Venues from '2nd Most Common Venue'
pizza_venues = pizza_venues[pizza_venues['2nd Most Common Venue'] !='American Restaurant']
pizza_venues = pizza_venues[pizza_venues['2nd Most Common Venue'] !='BBQ Joint']
pizza_venues = pizza_venues[pizza_venues['2nd Most Common Venue'] !='Burger Joint']
pizza_venues = pizza_venues[pizza_venues['2nd Most Common Venue'] !='Chinese Restaurant']
pizza_venues = pizza_venues[pizza_venues['2nd Most Common Venue'] !='Italian Restaurant']
pizza_venues = pizza_venues[pizza_venues['2nd Most Common Venue'] !='Latin American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['2nd Most Common Venue'] !='New American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['2nd Most Common Venue'] !='Sandwich Place']
pizza_venues = pizza_venues[pizza_venues['2nd Most Common Venue'] !='Sports Bar']
pizza_venues = pizza_venues[pizza_venues['2nd Most Common Venue'] !='Steakhouse']

#Drop Competeing Venues from '3rd Most Common Venue'
pizza_venues = pizza_venues[pizza_venues['3rd Most Common Venue'] !='American Restaurant']
pizza_venues = pizza_venues[pizza_venues['3rd Most Common Venue'] !='BBQ Joint']
pizza_venues = pizza_venues[pizza_venues['3rd Most Common Venue'] !='Burger Joint']
pizza_venues = pizza_venues[pizza_venues['3rd Most Common Venue'] !='Chinese Restaurant']
pizza_venues = pizza_venues[pizza_venues['3rd Most Common Venue'] !='Italian Restaurant']
pizza_venues = pizza_venues[pizza_venues['3rd Most Common Venue'] !='Latin American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['3rd Most Common Venue'] !='New American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['3rd Most Common Venue'] !='Sandwich Place']
pizza_venues = pizza_venues[pizza_venues['3rd Most Common Venue'] !='Sports Bar']
pizza_venues = pizza_venues[pizza_venues['3rd Most Common Venue'] !='Steakhouse']

#Drop Competeing Venues from '4th Most Common Venue'
pizza_venues = pizza_venues[pizza_venues['4th Most Common Venue'] !='American Restaurant']
pizza_venues = pizza_venues[pizza_venues['4th Most Common Venue'] !='BBQ Joint']
pizza_venues = pizza_venues[pizza_venues['4th Most Common Venue'] !='Burger Joint']
pizza_venues = pizza_venues[pizza_venues['4th Most Common Venue'] !='Chinese Restaurant']
pizza_venues = pizza_venues[pizza_venues['4th Most Common Venue'] !='Italian Restaurant']
pizza_venues = pizza_venues[pizza_venues['4th Most Common Venue'] !='Latin American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['4th Most Common Venue'] !='New American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['4th Most Common Venue'] !='Sandwich Place']
pizza_venues = pizza_venues[pizza_venues['4th Most Common Venue'] !='Sports Bar']
pizza_venues = pizza_venues[pizza_venues['4th Most Common Venue'] !='Steakhouse']

#Drop Competeing Venues from '5th Most Common Venue'
pizza_venues = pizza_venues[pizza_venues['5th Most Common Venue'] !='American Restaurant']
pizza_venues = pizza_venues[pizza_venues['5th Most Common Venue'] !='BBQ Joint']
pizza_venues = pizza_venues[pizza_venues['5th Most Common Venue'] !='Burger Joint']
pizza_venues = pizza_venues[pizza_venues['5th Most Common Venue'] !='Chinese Restaurant']
pizza_venues = pizza_venues[pizza_venues['5th Most Common Venue'] !='Italian Restaurant']
pizza_venues = pizza_venues[pizza_venues['5th Most Common Venue'] !='Latin American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['5th Most Common Venue'] !='New American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['5th Most Common Venue'] !='Sandwich Place']
pizza_venues = pizza_venues[pizza_venues['5th Most Common Venue'] !='Sports Bar']
pizza_venues = pizza_venues[pizza_venues['5th Most Common Venue'] !='Steakhouse']

#Drop Competeing Venues from '6th Most Common Venue'
pizza_venues = pizza_venues[pizza_venues['6th Most Common Venue'] !='American Restaurant']
pizza_venues = pizza_venues[pizza_venues['6th Most Common Venue'] !='BBQ Joint']
pizza_venues = pizza_venues[pizza_venues['6th Most Common Venue'] !='Burger Joint']
pizza_venues = pizza_venues[pizza_venues['6th Most Common Venue'] !='Chinese Restaurant']
pizza_venues = pizza_venues[pizza_venues['6th Most Common Venue'] !='Italian Restaurant']
pizza_venues = pizza_venues[pizza_venues['6th Most Common Venue'] !='Latin American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['6th Most Common Venue'] !='New American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['6th Most Common Venue'] !='Sandwich Place']
pizza_venues = pizza_venues[pizza_venues['6th Most Common Venue'] !='Sports Bar']
pizza_venues = pizza_venues[pizza_venues['6th Most Common Venue'] !='Steakhouse']

#Drop Competeing Venues from '7th Most Common Venue'
pizza_venues = pizza_venues[pizza_venues['7th Most Common Venue'] !='American Restaurant']
pizza_venues = pizza_venues[pizza_venues['7th Most Common Venue'] !='BBQ Joint']
pizza_venues = pizza_venues[pizza_venues['7th Most Common Venue'] !='Burger Joint']
pizza_venues = pizza_venues[pizza_venues['7th Most Common Venue'] !='Chinese Restaurant']
pizza_venues = pizza_venues[pizza_venues['7th Most Common Venue'] !='Italian Restaurant']
pizza_venues = pizza_venues[pizza_venues['7th Most Common Venue'] !='Latin American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['7th Most Common Venue'] !='New American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['7th Most Common Venue'] !='Sandwich Place']
pizza_venues = pizza_venues[pizza_venues['7th Most Common Venue'] !='Sports Bar']
pizza_venues = pizza_venues[pizza_venues['7th Most Common Venue'] !='Steakhouse']

#Drop Competeing Venues from '8th Most Common Venue'
pizza_venues = pizza_venues[pizza_venues['8th Most Common Venue'] !='American Restaurant']
pizza_venues = pizza_venues[pizza_venues['8th Most Common Venue'] !='BBQ Joint']
pizza_venues = pizza_venues[pizza_venues['8th Most Common Venue'] !='Burger Joint']
pizza_venues = pizza_venues[pizza_venues['8th Most Common Venue'] !='Chinese Restaurant']
pizza_venues = pizza_venues[pizza_venues['8th Most Common Venue'] !='Italian Restaurant']
pizza_venues = pizza_venues[pizza_venues['8th Most Common Venue'] !='Latin American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['8th Most Common Venue'] !='New American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['8th Most Common Venue'] !='Sandwich Place']
pizza_venues = pizza_venues[pizza_venues['8th Most Common Venue'] !='Sports Bar']
pizza_venues = pizza_venues[pizza_venues['8th Most Common Venue'] !='Steakhouse']

#Drop Competeing Venues from '9th Most Common Venue'
pizza_venues = pizza_venues[pizza_venues['9th Most Common Venue'] !='American Restaurant']
pizza_venues = pizza_venues[pizza_venues['9th Most Common Venue'] !='BBQ Joint']
pizza_venues = pizza_venues[pizza_venues['9th Most Common Venue'] !='Burger Joint']
pizza_venues = pizza_venues[pizza_venues['9th Most Common Venue'] !='Chinese Restaurant']
pizza_venues = pizza_venues[pizza_venues['9th Most Common Venue'] !='Italian Restaurant']
pizza_venues = pizza_venues[pizza_venues['9th Most Common Venue'] !='Latin American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['9th Most Common Venue'] !='New American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['9th Most Common Venue'] !='Sandwich Place']
pizza_venues = pizza_venues[pizza_venues['9th Most Common Venue'] !='Sports Bar']
pizza_venues = pizza_venues[pizza_venues['9th Most Common Venue'] !='Steakhouse']

#Drop Competeing Venues from '10th Most Common Venue'
pizza_venues = pizza_venues[pizza_venues['10th Most Common Venue'] !='American Restaurant']
pizza_venues = pizza_venues[pizza_venues['10th Most Common Venue'] !='BBQ Joint']
pizza_venues = pizza_venues[pizza_venues['10th Most Common Venue'] !='Burger Joint']
pizza_venues = pizza_venues[pizza_venues['10th Most Common Venue'] !='Chinese Restaurant']
pizza_venues = pizza_venues[pizza_venues['10th Most Common Venue'] !='Italian Restaurant']
pizza_venues = pizza_venues[pizza_venues['10th Most Common Venue'] !='Latin American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['10th Most Common Venue'] !='New American Restaraunt']
pizza_venues = pizza_venues[pizza_venues['10th Most Common Venue'] !='Sandwich Place']
pizza_venues = pizza_venues[pizza_venues['10th Most Common Venue'] !='Sports Bar']
pizza_venues = pizza_venues[pizza_venues['10th Most Common Venue'] !='Steakhouse']

In [53]:
pizza_venues.shape

(32, 11)

In [55]:
pizza_venues.head(32)

,Neighborhood name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Ballard,French Restaurant,Bus Stop,Park,Jewelry Store,Coffee Shop,Women's Store,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
3,Beacon Hill,Bus Station,Light Rail Station,Women's Store,Eye Doctor,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fair
4,Belltown,Bus Stop,Harbor / Marina,Boat or Ferry,Intersection,Deli / Bodega,Dive Shop,Bus Line,Park,Trail,Grocery Store
8,Broadmoor,Soccer Field,Playground,Garden,Golf Course,Ethiopian Restaurant,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
9,Broadview,Concert Hall,Trail,Dog Run,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
10,Bryant,Trail,Farm,Baseball Field,Women's Store,Event Space,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
13,Cedar Park,Supermarket,Vietnamese Restaurant,Women's Store,Eye Doctor,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
14,Central Seattle,Ethiopian Restaurant,Thai Restaurant,Asian Restaurant,Bar,Playground,Baseball Field,Gym Pool,Deli / Bodega,Bus Stop,Grocery Store
15,Central Waterfront,Coffee Shop,Seafood Restaurant,Hotel,Bakery,French Restaurant,Cocktail Bar,Brewery,Breakfast Spot,Hobby Shop,Herbs & Spices Store
25,Fauntleroy,Fish & Chips Shop,Botanical Garden,Pilates Studio,Coffee Shop,Cupcake Shop,Deli / Bodega,Park,Cosmetics Shop,Gym / Fitness Center,Eastern European Restaurant


Now we've narrowed down the list of over 100 neighborhoods to 32 neighborhoods where a new Pizza Place would likely succeed. This will be the end of the capstone project. This result gives us a strong baseline that could be analyzed into further detail depending on priorities of the business. Thank you for taking the time to review this project!